In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel('./number_HSS.xlsx', sheet_name='本地直连', index_col=None, dtype=str)

In [3]:
df = df.iloc[:, :4]

In [4]:
df

,省份,区号,E164,归属HLR
0,重庆,23,861500230,CQHSS03FE01
1,重庆,23,861500231,CQHSS03FE01
2,重庆,23,861500232,CQHSS03FE01
3,重庆,23,861500233,CQHSS03FE01
4,重庆,23,861500234,CQHSS03FE01
...,...,...,...,...
3063,重庆,23,861952205,CQHSS07FE01
3064,重庆,23,861952206,CQHSS07FE01
3065,重庆,23,861952207,CQHSS07FE01
3066,重庆,23,861952443,CQHSS07FE01


In [5]:
a_list = ['8615002301111', '8615002311111', '8615002312222', '8610002331111']

In [6]:
df2 = pd.DataFrame(a_list, columns=['msisdn'])
df2

,msisdn
0,8615002301111
1,8615002311111
2,8615002312222
3,8610002331111


In [7]:
df2['E164'] = df2['msisdn'].apply(lambda x: x[:9])

In [8]:
df2

,msisdn,E164
0,8615002301111,861500230
1,8615002311111,861500231
2,8615002312222,861500231
3,8610002331111,861000233


In [9]:
df3 = pd.merge(df2, df, how='left', on='E164')
df3

,msisdn,E164,省份,区号,归属HLR
0,8615002301111,861500230,重庆,23,CQHSS03FE01
1,8615002311111,861500231,重庆,23,CQHSS03FE01
2,8615002312222,861500231,重庆,23,CQHSS03FE01
3,8610002331111,861000233,NaN,NaN,NaN


In [10]:
df4 = df3.loc[:, ['msisdn', 'E164', '归属HLR']]
df4

,msisdn,E164,归属HLR
0,8615002301111,861500230,CQHSS03FE01
1,8615002311111,861500231,CQHSS03FE01
2,8615002312222,861500231,CQHSS03FE01
3,8610002331111,861000233,NaN


In [11]:
df5 = df4.fillna('unknown')
df5

,msisdn,E164,归属HLR
0,8615002301111,861500230,CQHSS03FE01
1,8615002311111,861500231,CQHSS03FE01
2,8615002312222,861500231,CQHSS03FE01
3,8610002331111,861000233,unknown


In [12]:
df5['归属HLR'].to_list()

['CQHSS03FE01', 'CQHSS03FE01', 'CQHSS03FE01', 'unknown']

In [13]:
def get_hss_df():
    hss_df = pd.read_excel('number_HSS.xlsx', sheet_name='本地直连', index_col=None, dtype=str)
    hss_df = hss_df.iloc[:, :4]
    hss_df = hss_df.rename(columns={
        '省份': 'province',
        '区号': 'areaCode',
        'E164': 'E164',
        '归属HLR': 'belongToHSS',
    })
    return hss_df

In [14]:
def MSISDN_format(msisdn):
    """
    返回以86开头的用户号码格式， 如果不符合msisdn格式则返回invalid

    Args:
        msisdn: (str)  需要分析的原始号码

    Returns: 规整后的号码

    """
    if msisdn == '' or msisdn is None:
        return 'invalid'
    p = re.compile(r'^(86)?\d{11}$')
    try:
        m = p.match(str(msisdn).strip())
    except Exception as e:
        print(f"MSISDN format msisdn: {msisdn} have error: {e}")
        return 'invalid'
    if m is None:
        return 'invalid'
    num = m.group()
    if num.startswith('86'):
        return num
    return f'86{num}'

In [15]:
resource_df = pd.read_csv('./444.txt', names=['PhoneNum'], dtype=str)
resource_df

,PhoneNum
0,15922730211
1,15923157014
2,15223769474
3,15025451665
4,18896159969
...,...
86,13896304363
87,15823711962
88,17823785398
89,13983857067


In [16]:
resource_df['PhoneNum'] = resource_df['PhoneNum'].map(MSISDN_format)
resource_df

,PhoneNum
0,8615922730211
1,8615923157014
2,8615223769474
3,8615025451665
4,8618896159969
...,...
86,8613896304363
87,8615823711962
88,8617823785398
89,8613983857067


In [17]:
resource_df = resource_df[~resource_df['PhoneNum'].str.contains('invalid')]
resource_df

,PhoneNum
0,8615922730211
1,8615923157014
2,8615223769474
3,8615025451665
4,8618896159969
...,...
86,8613896304363
87,8615823711962
88,8617823785398
89,8613983857067


In [18]:
resource_df.drop_duplicates('PhoneNum', keep='first', inplace=True)
resource_df

,PhoneNum
0,8615922730211
1,8615923157014
2,8615223769474
3,8615025451665
4,8618896159969
...,...
86,8613896304363
87,8615823711962
88,8617823785398
89,8613983857067


In [19]:
resource_df['E164'] = resource_df['PhoneNum'].apply(lambda x: x[:9])
resource_df

,PhoneNum,E164
0,8615922730211,861592273
1,8615923157014,861592315
2,8615223769474,861522376
3,8615025451665,861502545
4,8618896159969,861889615
...,...,...
86,8613896304363,861389630
87,8615823711962,861582371
88,8617823785398,861782378
89,8613983857067,861398385


In [20]:
hss_df = get_hss_df()
hss_df

,province,areaCode,E164,belongToHSS
0,重庆,23,861500230,CQHSS03FE01
1,重庆,23,861500231,CQHSS03FE01
2,重庆,23,861500232,CQHSS03FE01
3,重庆,23,861500233,CQHSS03FE01
4,重庆,23,861500234,CQHSS03FE01
...,...,...,...,...
3063,重庆,23,861952205,CQHSS07FE01
3064,重庆,23,861952206,CQHSS07FE01
3065,重庆,23,861952207,CQHSS07FE01
3066,重庆,23,861952443,CQHSS07FE01


In [21]:
resource_df = pd.merge(resource_df, hss_df, how='left', on='E164')
resource_df

,PhoneNum,E164,province,areaCode,belongToHSS
0,8615922730211,861592273,重庆,23,CQHSS04FE02
1,8615923157014,861592315,重庆,23,CQHSS05FE02
2,8615223769474,861522376,重庆,23,CQHSS03FE02
3,8615025451665,861502545,重庆,23,CQHSS03FE01
4,8618896159969,861889615,NaN,NaN,NaN
...,...,...,...,...,...
86,8613896304363,861389630,重庆,23,CQHSS06FE02
87,8615823711962,861582371,重庆,23,CQHSS04FE02
88,8617823785398,861782378,NaN,NaN,NaN
89,8613983857067,861398385,重庆,23,CQHSS05FE02


In [22]:
pd.set_option('display.max_rows', 500)

resource_df = resource_df.fillna('unknown')
resource_df

,PhoneNum,E164,province,areaCode,belongToHSS
0,8615922730211,861592273,重庆,23,CQHSS04FE02
1,8615923157014,861592315,重庆,23,CQHSS05FE02
2,8615223769474,861522376,重庆,23,CQHSS03FE02
3,8615025451665,861502545,重庆,23,CQHSS03FE01
4,8618896159969,861889615,unknown,unknown,unknown
5,8615023916919,861502391,重庆,23,CQHSS03FE01
6,8615086637082,861508663,重庆,23,CQHSS03FE01
7,8613647647595,861364764,重庆,23,CQHSS04FE01
8,8615213616672,861521361,重庆,23,CQHSS03FE02
9,8615723666619,861572366,重庆,23,CQHSS06FE02


In [23]:
toHSS = resource_df.loc[:, ['PhoneNum', 'belongToHSS']]
toHSS

,PhoneNum,belongToHSS
0,8615922730211,CQHSS04FE02
1,8615923157014,CQHSS05FE02
2,8615223769474,CQHSS03FE02
3,8615025451665,CQHSS03FE01
4,8618896159969,unknown
5,8615023916919,CQHSS03FE01
6,8615086637082,CQHSS03FE01
7,8613647647595,CQHSS04FE01
8,8615213616672,CQHSS03FE02
9,8615723666619,CQHSS06FE02


In [24]:
bth_count_df = toHSS.groupby('belongToHSS').count()
bth_count_df

,PhoneNum
belongToHSS,
CQHSS03FE01,8
CQHSS03FE02,7
CQHSS04FE01,9
CQHSS04FE02,8
CQHSS05FE01,8
CQHSS05FE02,13
CQHSS06FE01,4
CQHSS06FE02,8
unknown,26


In [25]:
HSSweidu1 = pd.DataFrame(bth_count_df)
HSSweidu1 = HSSweidu1.rename(columns={'PhoneNum':'count'})
HSSweidu1

,count
belongToHSS,
CQHSS03FE01,8
CQHSS03FE02,7
CQHSS04FE01,9
CQHSS04FE02,8
CQHSS05FE01,8
CQHSS05FE02,13
CQHSS06FE01,4
CQHSS06FE02,8
unknown,26


In [26]:
total = HSSweidu1['count'].sum()
total

91

In [27]:
HSSweidu1['zhanbi'] = HSSweidu1['count'].apply(lambda x: x / total * 100)
HSSweidu1

,count,zhanbi
belongToHSS,,
CQHSS03FE01,8,8.791209
CQHSS03FE02,7,7.692308
CQHSS04FE01,9,9.890110
CQHSS04FE02,8,8.791209
CQHSS05FE01,8,8.791209
CQHSS05FE02,13,14.285714
CQHSS06FE01,4,4.395604
CQHSS06FE02,8,8.791209
unknown,26,28.571429


In [28]:
HSSweidu1.sort_values(by='zhanbi', ascending=False)

,count,zhanbi
belongToHSS,,
unknown,26,28.571429
CQHSS05FE02,13,14.285714
CQHSS04FE01,9,9.890110
CQHSS03FE01,8,8.791209
CQHSS04FE02,8,8.791209
CQHSS05FE01,8,8.791209
CQHSS06FE02,8,8.791209
CQHSS03FE02,7,7.692308
CQHSS06FE01,4,4.395604


In [29]:
import os
HSSweidu1.to_csv(os.path.join('./', 'belongToHSS.csv'))

In [30]:
HSSweidu1.loc['unknown', 'count']

26

In [31]:
toHSS.groupby('belongToHSS')['PhoneNum'].apply(list).to_dict()

{'CQHSS03FE01': ['8615025451665',
  '8615023916919',
  '8615086637082',
  '8615023603533',
  '8615025648392',
  '8615025593512',
  '8615025473084',
  '8615023525354'],
 'CQHSS03FE02': ['8615223769474',
  '8615213616672',
  '8615213752090',
  '8615215162080',
  '8615213358922',
  '8615202393237',
  '8615213053636'],
 'CQHSS04FE01': ['8613647647595',
  '8613996196019',
  '8613883531717',
  '8613883834305',
  '8618423394515',
  '8618375648607',
  '8613883680652',
  '8613678491609',
  '8613883510002'],
 'CQHSS04FE02': ['8615922730211',
  '8615823274180',
  '8615922868915',
  '8615922789552',
  '8615922854686',
  '8615823083710',
  '8615922831773',
  '8615823711962'],
 'CQHSS05FE01': ['8613983975808',
  '8613637741523',
  '8613983908520',
  '8615909331999',
  '8613752814268',
  '8613896166768',
  '8613896195183',
  '8613896129911'],
 'CQHSS05FE02': ['8615923157014',
  '8613883197269',
  '8613608383070',
  '8613908323969',
  '8613527395750',
  '8615923504757',
  '8613527377146',
  '861398307

In [32]:
resultDict = [{'8613527541191': {'userType': 'volte', 'mmeaddr': 'mmec72.mmegi03a1.mme.epc.mnc000.mcc460.3gppnetwork.org\r', 'vlraddr': '8613442965', 'pgwaddr': 'ims$topon.pgw-s5.cqsaegw39ber.cq.cq._cmwap$topon.pgw-s5.cqsaegw12bhw.cq.cq._cmnet$topon.pgw-s5.cqsaegw38ber.cq.cq.', 'scscfaddr': 'cqscscf14bhw.cq'}}, {'8615023642351': {'userType': 'volte', 'mmeaddr': 'mmec6E.mmegi03A1.mme.epc.mnc000.mcc460.3gppnetwork.org\r', 'vlraddr': '8613441966', 'pgwaddr': 'cmnet$topon.pgw-s5.cqsaegw34ber.cq.cq.', 'scscfaddr': 'unknown'}}]

In [33]:
result_df = pd.DataFrame(resultDict).T
result_df


,0,1
8613527541191,"{'userType': 'volte', 'mmeaddr': 'mmec72.mmegi...",NaN
8615023642351,NaN,"{'userType': 'volte', 'mmeaddr': 'mmec6E.mmegi..."


In [34]:
fxjg_df = pd.read_csv('./fenxijieguo.csv')
g_df = fxjg_df.groupby('userType').count().iloc[:, :1]
g_df

,Unnamed: 0
userType,
volte,4


In [35]:
g_df = g_df.rename({g_df.columns[0]: 'count'}, axis=1)
g_df

,count
userType,
volte,4


In [36]:
total = g_df['count'].sum()
total

4

In [37]:
g_df['zhanbi'] = g_df['count'].apply(lambda x: x / total * 100)
g_df.sort_values('count', ascending=False)


,count,zhanbi
userType,,
volte,4,100.0


In [38]:
MSC_ER_INFO_PATH = './msc_er_info.xlsx'
MSC_HW_INFO_PATH = './msc_hw_info.xlsx'

In [39]:
msc_er_info_df = pd.read_excel(MSC_ER_INFO_PATH, sheet_name='msc_er_info', index_col=None, dtype=str)
msc_er_info_df = msc_er_info_df.loc[:, ['msc_id', 'msc_name', 'msc_location', 'mscpool']]
msc_er_info_df = msc_er_info_df.rename(columns={
    'msc_name': 'vlraddr',
    'msc_location': 'vlrlocation',
    'mscpool': 'mscPooladdr',
})

In [40]:
msc_er_info_df

,msc_id,vlraddr,vlrlocation,mscPooladdr
0,8613441963,CQGS04,人和19楼,POOL4
1,8613441966,CQGS07,IDC5楼,POOL2
2,8613441967,CQGS08,IDC5楼,POOL3
3,8613441968,CQGS09,渝北4楼,POOL5
4,8613441969,CQGS10,人和20楼,POOL5
5,8613442961,CQGS12,人和19楼,POOL4
6,8613442962,CQGS13,人和20楼,POOL2
7,8613442965,CQGS16,人和20楼,POOL2
8,8613442966,CQGS17,IDC13楼,POOL2
9,8613442967,CQGS18,IDC13楼,POOL3


In [41]:
msc_hw_info_df = pd.read_excel(MSC_HW_INFO_PATH, sheet_name='msc_hw_info', index_col=None, dtype=str)
msc_hw_info_df = msc_hw_info_df.loc[:, ['msc_id', 'msc_name', 'msc_location', 'msc_pool']]
msc_hw_info_df = msc_hw_info_df.rename(columns={
    'msc_name': 'vlraddr',
    'msc_location': 'vlrlocation',
    'msc_pool': 'mscPooladdr',
})

In [42]:
msc_hw_info_df

,msc_id,vlraddr,vlrlocation,mscPooladdr
0,8613442964,CQGS15,中国重庆万州观音岩综合楼202机房交换1,万州POOL1
1,8613443961,CQGS22,中国重庆万州观音岩综合楼203机房交换2,万州POOL1
2,8613444961,CQGS32,中国重庆万州天城大道综合楼2楼交换机房,万州POOL1
3,8613444962,CQGS33,中国重庆万州观音岩综合楼203机房交换2,万州POOL1
4,8613449960,CQGS35,中国重庆万州观音岩综合楼206机房交换3,万州POOL1
5,8613444966,CQGS37,中国重庆万州观音岩综合楼206机房交换3,万州POOL1
6,8613445963,CQGS44,中国重庆万州天城大道综合楼2楼机房交换,万州POOL1
7,8613445968,CQGS49,中国重庆万州观音岩综合楼303机房交换,万州POOL1
8,8613449961,CQGS94,中国重庆万州观音岩综合楼203机房交换2,万州POOL1
9,8613443962,CQGS23,中国重庆重庆IDC901机房交换1,主城POOL6


In [43]:
pd.concat([msc_er_info_df, msc_hw_info_df]).reset_index(drop=True)

,msc_id,vlraddr,vlrlocation,mscPooladdr
0,8613441963,CQGS04,人和19楼,POOL4
1,8613441966,CQGS07,IDC5楼,POOL2
2,8613441967,CQGS08,IDC5楼,POOL3
3,8613441968,CQGS09,渝北4楼,POOL5
4,8613441969,CQGS10,人和20楼,POOL5
5,8613442961,CQGS12,人和19楼,POOL4
6,8613442962,CQGS13,人和20楼,POOL2
7,8613442965,CQGS16,人和20楼,POOL2
8,8613442966,CQGS17,IDC13楼,POOL2
9,8613442967,CQGS18,IDC13楼,POOL3


In [44]:
df1 = pd.read_csv('fenxijieguo.csv', index_col=0)
df1

,belongToHSS,userType,vlraddr,mscPooladdr,vlrlocation,mmeaddr,mmePooladdr,cmnet_pgwaddr,ims_pgwaddr,scscfaddr
8615922730211,CQHSS04FE02BER,volte,8613441963,POOL3,IDC5楼,mmec6A.mmegi03A3,CQMME-POOL3,cqsaegw40ber.cq,cqsaegw40ber.cq,app-xncdcqscscf1bzx-04azx012.cq
8615923157014,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec42.mmegi03A3,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq
8615923157015,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec6E.mmegi03A1,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq
8615923157016,CQHSS05FE02BER,volte,86134419999,POOL2,IDC5楼,mmec6E.mmegi03A1111,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq


In [45]:
df2 = df1.rename(columns={'vlraddr': 'msc_id'})
df2 = df2.astype(str)
df2

,belongToHSS,userType,msc_id,mscPooladdr,vlrlocation,mmeaddr,mmePooladdr,cmnet_pgwaddr,ims_pgwaddr,scscfaddr
8615922730211,CQHSS04FE02BER,volte,8613441963,POOL3,IDC5楼,mmec6A.mmegi03A3,CQMME-POOL3,cqsaegw40ber.cq,cqsaegw40ber.cq,app-xncdcqscscf1bzx-04azx012.cq
8615923157014,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec42.mmegi03A3,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq
8615923157015,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec6E.mmegi03A1,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq
8615923157016,CQHSS05FE02BER,volte,86134419999,POOL2,IDC5楼,mmec6E.mmegi03A1111,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq


In [46]:
for key in ['vlraddr', 'vlrlocation', 'mscPooladdr']:
    if key in df2.columns:
        df2 = df2.drop(key, axis=1)
# df2 = df2.drop('vlrlocation', axis=1)

pd.merge(df2, msc_er_info_df, how='left', on='msc_id')

,belongToHSS,userType,msc_id,mmeaddr,mmePooladdr,cmnet_pgwaddr,ims_pgwaddr,scscfaddr,vlraddr,vlrlocation,mscPooladdr
0,CQHSS04FE02BER,volte,8613441963,mmec6A.mmegi03A3,CQMME-POOL3,cqsaegw40ber.cq,cqsaegw40ber.cq,app-xncdcqscscf1bzx-04azx012.cq,CQGS04,人和19楼,POOL4
1,CQHSS05FE02BER,volte,8613441966,mmec42.mmegi03A3,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq,CQGS07,IDC5楼,POOL2
2,CQHSS05FE02BER,volte,8613441966,mmec6E.mmegi03A1,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq,CQGS07,IDC5楼,POOL2
3,CQHSS05FE02BER,volte,86134419999,mmec6E.mmegi03A1111,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq,NaN,NaN,NaN


In [47]:
MME_INFO_PATH = './mme_info.xlsx'
mme_info_df = pd.read_excel(MME_INFO_PATH, sheet_name='mme_info', index_col=None, dtype=str)
mme_info_df['mme_id'] = "mmec" + mme_info_df['mmec'] + ".mmegi" + mme_info_df['mmegi']
mme_info_df = mme_info_df.loc[:, ['mme_name', 'mmepool_name', 'mme_id']]
mme_info_df = mme_info_df.rename(columns={
    'mme_name': 'mmeaddr',
    'mmepool_name': 'mmePooladdr'
})
mme_info_df

,mmeaddr,mmePooladdr,mme_id
0,CQMME05BER,CQMME-POOL2,mmec6A.mmegi03A3
1,CQMME08BER,CQMME-POOL2,mmec40.mmegi03A3
2,CQMME13BER,CQMME-POOL2,mmec42.mmegi03A3
3,CQMME25BER,CQMME-POOL2,mmec4A.mmegi03A3
4,CQMME01BER,CQMME-POOL1,mmec70.mmegi03A1
5,CQMME02BER,CQMME-POOL1,mmec72.mmegi03A1
6,CQMME06BER,CQMME-POOL1,mmec74.mmegi03A1
7,CQMME07BER,CQMME-POOL1,mmec76.mmegi03A1
8,CQMME14BER,CQMME-POOL1,mmec6E.mmegi03A1
9,CQMME19BER,CQMME-POOL1,mmec44.mmegi03A1


In [48]:
df1 = pd.read_csv('fenxijieguo.csv', index_col=0)
df1

,belongToHSS,userType,vlraddr,mscPooladdr,vlrlocation,mmeaddr,mmePooladdr,cmnet_pgwaddr,ims_pgwaddr,scscfaddr
8615922730211,CQHSS04FE02BER,volte,8613441963,POOL3,IDC5楼,mmec6A.mmegi03A3,CQMME-POOL3,cqsaegw40ber.cq,cqsaegw40ber.cq,app-xncdcqscscf1bzx-04azx012.cq
8615923157014,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec42.mmegi03A3,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq
8615923157015,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec6E.mmegi03A1,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq
8615923157016,CQHSS05FE02BER,volte,86134419999,POOL2,IDC5楼,mmec6E.mmegi03A1111,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq


In [49]:
df1 = df1.rename(columns={'mmeaddr': 'mme_id'})  # 重命名mmeaddr为mme_id
df1

,belongToHSS,userType,vlraddr,mscPooladdr,vlrlocation,mme_id,mmePooladdr,cmnet_pgwaddr,ims_pgwaddr,scscfaddr
8615922730211,CQHSS04FE02BER,volte,8613441963,POOL3,IDC5楼,mmec6A.mmegi03A3,CQMME-POOL3,cqsaegw40ber.cq,cqsaegw40ber.cq,app-xncdcqscscf1bzx-04azx012.cq
8615923157014,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec42.mmegi03A3,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq
8615923157015,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec6E.mmegi03A1,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq
8615923157016,CQHSS05FE02BER,volte,86134419999,POOL2,IDC5楼,mmec6E.mmegi03A1111,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq


In [50]:
for key in ['mmeaddr', 'mmePooladdr']:
    if key in df1.columns:
        df1 = df1.drop(key, axis=1)  # 有就删除之前的对应字段数据
df1

,belongToHSS,userType,vlraddr,mscPooladdr,vlrlocation,mme_id,cmnet_pgwaddr,ims_pgwaddr,scscfaddr
8615922730211,CQHSS04FE02BER,volte,8613441963,POOL3,IDC5楼,mmec6A.mmegi03A3,cqsaegw40ber.cq,cqsaegw40ber.cq,app-xncdcqscscf1bzx-04azx012.cq
8615923157014,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec42.mmegi03A3,unknown,unknown,cqscscf13bhw.cq
8615923157015,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec6E.mmegi03A1,unknown,unknown,cqscscf13bhw.cq
8615923157016,CQHSS05FE02BER,volte,86134419999,POOL2,IDC5楼,mmec6E.mmegi03A1111,unknown,unknown,cqscscf13bhw.cq


In [51]:
df2 = pd.merge(df1, mme_info_df, how='left', on='mme_id')  # 进行数据合并新增对应的列
# df2 = df2.fillna('unknown')
df2.loc[pd.isna(df2['mmeaddr']), 'mmeaddr'] = df2.loc[pd.isna(df2['mmeaddr']), 'mme_id']
df2

,belongToHSS,userType,vlraddr,mscPooladdr,vlrlocation,mme_id,cmnet_pgwaddr,ims_pgwaddr,scscfaddr,mmeaddr,mmePooladdr
0,CQHSS04FE02BER,volte,8613441963,POOL3,IDC5楼,mmec6A.mmegi03A3,cqsaegw40ber.cq,cqsaegw40ber.cq,app-xncdcqscscf1bzx-04azx012.cq,CQMME05BER,CQMME-POOL2
1,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec42.mmegi03A3,unknown,unknown,cqscscf13bhw.cq,CQMME13BER,CQMME-POOL2
2,CQHSS05FE02BER,volte,8613441966,POOL2,IDC5楼,mmec6E.mmegi03A1,unknown,unknown,cqscscf13bhw.cq,CQMME14BER,CQMME-POOL1
3,CQHSS05FE02BER,volte,86134419999,POOL2,IDC5楼,mmec6E.mmegi03A1111,unknown,unknown,cqscscf13bhw.cq,mmec6E.mmegi03A1111,NaN


In [52]:
df3 = df2.loc[df2['vlraddr'] == 86134419999, :]
df3


,belongToHSS,userType,vlraddr,mscPooladdr,vlrlocation,mme_id,cmnet_pgwaddr,ims_pgwaddr,scscfaddr,mmeaddr,mmePooladdr
3,CQHSS05FE02BER,volte,86134419999,POOL2,IDC5楼,mmec6E.mmegi03A1111,unknown,unknown,cqscscf13bhw.cq,mmec6E.mmegi03A1111,NaN


In [53]:
df4 = df2.iloc[:, :1]
df4

,belongToHSS
0,CQHSS04FE02BER
1,CQHSS05FE02BER
2,CQHSS05FE02BER
3,CQHSS05FE02BER


In [54]:
df4['mme_id'] = df3['mme_id']
df4

,belongToHSS,mme_id
0,CQHSS04FE02BER,NaN
1,CQHSS05FE02BER,NaN
2,CQHSS05FE02BER,NaN
3,CQHSS05FE02BER,mmec6E.mmegi03A1111


In [55]:
df5 = pd.read_csv('./fenxijieguo-Copy1.csv', index_col=0)
df5

,belongToHSS,userType,vlraddr,mscPooladdr,vlrlocation,mmeaddr,mmePooladdr,cmnet_pgwaddr,ims_pgwaddr,scscfaddr
8615922730211,CQHSS04FE02BER,volte,重庆GS08,POOL3,IDC5楼,CQMME09BHW,CQMME-POOL3,cqsaegw40ber.cq,cqsaegw40ber.cq,app-xncdcqscscf1bzx-04azx012.cq
8615923157014,CQHSS05FE02BER,volte,重庆GS07,POOL2,IDC5楼,CQMME15BHW,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq
8615223769474,CQHSS03FE02BER,volte,重庆GS19,POOL4,渝北4楼,CQMME07BER,CQMME-POOL1,cqsaegw33ber.cq,cqsaegw33ber.cq,cqscscf8bzx.cq
8615025451665,CQHSS03FE01BER,volte,重庆GS41,POOL4,人和20楼,CQMME06BER,CQMME-POOL1,cqsaegw37ber.cq,cqsaegw37ber.cq,cqscscf9bzx.cq
8618896159969,CQHSS07FE02BHW,volte,unknown,unknown,unknown,mmec82.mmegi0323,unknown,whsaegw78ber.wh,whsaegw78ber.wh,cqscscf14bhw.cq
8615023916919,CQHSS03FE01BER,volte,重庆GS50,涪陵POOL7,中国重庆涪陵桥南综合楼5楼机房交换2,CQMME05BER,CQMME-POOL2,cqsaegw35ber.cq,cqsaegw35ber.cq,unknown
8615086637082,CQHSS03FE01BER,volte,重庆GS23,主城POOL6,中国重庆重庆IDC901机房交换1,CQMME19BER,CQMME-POOL1,unknown,unknown,cqscscf5bzx.cq
8613647647595,CQHSS04FE01BER,volte,重庆GS27,POOL4,毛线沟3楼,CQMME19BER,CQMME-POOL1,cqsaegw38ber.cq,cqsaegw38ber.cq,cqscscf8bzx.cq
8615213616672,CQHSS03FE02BER,volte,重庆GS48,涪陵POOL7,中国重庆涪陵桥南综合楼4楼机房交换1,CQMME25BER,CQMME-POOL2,cqsaegw27ber.cq,cqsaegw27ber.cq,cqscscf6bzx.cq
8615723666619,CQHSS06FE02BER,volte,重庆GS20,POOL5,渝北4楼,CQMME07BER,CQMME-POOL1,cqsaegw30ber.cq,cqsaegw31ber.cq,cqscscf5bzx.cq


In [56]:
df6 = df5[df5['scscfaddr'].notna()]
df6 = df6[df6['scscfaddr'].str.contains(r'.*?\.cq')]
df6

,belongToHSS,userType,vlraddr,mscPooladdr,vlrlocation,mmeaddr,mmePooladdr,cmnet_pgwaddr,ims_pgwaddr,scscfaddr
8615922730211,CQHSS04FE02BER,volte,重庆GS08,POOL3,IDC5楼,CQMME09BHW,CQMME-POOL3,cqsaegw40ber.cq,cqsaegw40ber.cq,app-xncdcqscscf1bzx-04azx012.cq
8615923157014,CQHSS05FE02BER,volte,重庆GS07,POOL2,IDC5楼,CQMME15BHW,CQMME-POOL3,unknown,unknown,cqscscf13bhw.cq
8615223769474,CQHSS03FE02BER,volte,重庆GS19,POOL4,渝北4楼,CQMME07BER,CQMME-POOL1,cqsaegw33ber.cq,cqsaegw33ber.cq,cqscscf8bzx.cq
8615025451665,CQHSS03FE01BER,volte,重庆GS41,POOL4,人和20楼,CQMME06BER,CQMME-POOL1,cqsaegw37ber.cq,cqsaegw37ber.cq,cqscscf9bzx.cq
8618896159969,CQHSS07FE02BHW,volte,unknown,unknown,unknown,mmec82.mmegi0323,unknown,whsaegw78ber.wh,whsaegw78ber.wh,cqscscf14bhw.cq
8615086637082,CQHSS03FE01BER,volte,重庆GS23,主城POOL6,中国重庆重庆IDC901机房交换1,CQMME19BER,CQMME-POOL1,unknown,unknown,cqscscf5bzx.cq
8613647647595,CQHSS04FE01BER,volte,重庆GS27,POOL4,毛线沟3楼,CQMME19BER,CQMME-POOL1,cqsaegw38ber.cq,cqsaegw38ber.cq,cqscscf8bzx.cq
8615213616672,CQHSS03FE02BER,volte,重庆GS48,涪陵POOL7,中国重庆涪陵桥南综合楼4楼机房交换1,CQMME25BER,CQMME-POOL2,cqsaegw27ber.cq,cqsaegw27ber.cq,cqscscf6bzx.cq
8615723666619,CQHSS06FE02BER,volte,重庆GS20,POOL5,渝北4楼,CQMME07BER,CQMME-POOL1,cqsaegw30ber.cq,cqsaegw31ber.cq,cqscscf5bzx.cq
8615023603533,CQHSS03FE01BER,volte,重庆GS31,POOL4,渝北4楼,CQMME14BER,CQMME-POOL1,cqsaegw14ber.cq,cqsaegw14ber.cq,cqscscf8bzx.cq


In [57]:
zte_df =df6[df6['scscfaddr'].str.contains(r'.*?zx\.cq')]
zte_df

,belongToHSS,userType,vlraddr,mscPooladdr,vlrlocation,mmeaddr,mmePooladdr,cmnet_pgwaddr,ims_pgwaddr,scscfaddr
8615223769474,CQHSS03FE02BER,volte,重庆GS19,POOL4,渝北4楼,CQMME07BER,CQMME-POOL1,cqsaegw33ber.cq,cqsaegw33ber.cq,cqscscf8bzx.cq
8615025451665,CQHSS03FE01BER,volte,重庆GS41,POOL4,人和20楼,CQMME06BER,CQMME-POOL1,cqsaegw37ber.cq,cqsaegw37ber.cq,cqscscf9bzx.cq
8615086637082,CQHSS03FE01BER,volte,重庆GS23,主城POOL6,中国重庆重庆IDC901机房交换1,CQMME19BER,CQMME-POOL1,unknown,unknown,cqscscf5bzx.cq
8613647647595,CQHSS04FE01BER,volte,重庆GS27,POOL4,毛线沟3楼,CQMME19BER,CQMME-POOL1,cqsaegw38ber.cq,cqsaegw38ber.cq,cqscscf8bzx.cq
8615213616672,CQHSS03FE02BER,volte,重庆GS48,涪陵POOL7,中国重庆涪陵桥南综合楼4楼机房交换1,CQMME25BER,CQMME-POOL2,cqsaegw27ber.cq,cqsaegw27ber.cq,cqscscf6bzx.cq
8615723666619,CQHSS06FE02BER,volte,重庆GS20,POOL5,渝北4楼,CQMME07BER,CQMME-POOL1,cqsaegw30ber.cq,cqsaegw31ber.cq,cqscscf5bzx.cq
8615023603533,CQHSS03FE01BER,volte,重庆GS31,POOL4,渝北4楼,CQMME14BER,CQMME-POOL1,cqsaegw14ber.cq,cqsaegw14ber.cq,cqscscf8bzx.cq
8615823274180,CQHSS04FE02BER,volte,重庆GS29,主城POOL6,中国重庆重庆南坪505机房交换2,CQMME04BHW,CQMME-POOL3,unknown,cqsaegw38ber.cq,cqscscf7bzx.cq
8613709468964,CQHSS06FE02BER,volte,重庆GS09,POOL5,渝北4楼,CQMME01BER,CQMME-POOL1,cqsaegw40ber.cq,cqsaegw40ber.cq,cqscscf8bzx.cq
8613628267979,CQHSS06FE02BER,volte,重庆GS43,涪陵POOL7,中国重庆涪陵桥南综合楼6楼机房交换3,CQMME13BER,CQMME-POOL2,cqsaegw41ber.cq,cqsaegw41ber.cq,cqscscf8bzx.cq


In [58]:
zte_g = zte_df.groupby('scscfaddr')
zte_dict = {}
for key, item in zte_g:
    zte_dict[key] = item.index.to_list()
zte_dict

{'cqscscf5bzx.cq': [8615086637082,
  8615723666619,
  8613996765557,
  8615723198155],
 'cqscscf6bzx.cq': [8615213616672, 8615025593512, 8615826267293],
 'cqscscf7bzx.cq': [8615823274180,
  8615213752090,
  8615025648392,
  8615823083710,
  8618423394515,
  8615213358922],
 'cqscscf8bzx.cq': [8615223769474,
  8613647647595,
  8615023603533,
  8613709468964,
  8613628267979,
  8615823941201,
  8615023525354,
  8615202393237,
  8613896304363],
 'cqscscf9bzx.cq': [8615025451665,
  8618723314386,
  8618723397033,
  8615025473084,
  8618723583873,
  8615213053636]}

In [59]:
SCSCF_ADDR_REG_DICT = {
    'zte': r'.*?zx\.cq$',
    'hw': r'.*?hw\.cq$',
    'NFV': r'^app.*?\.cq$',
}
def get_scscfaddr_dict(df):
    """

    Args:
        df:

    Returns:

    """
    temp_df = df[df['scscfaddr'].notna()]
    temp_df = temp_df[temp_df['scscfaddr'].str.contains(r'.*?\.cq')]  # 获取重庆的scscfaddr
    scscfaddr_dict = {}
    for k, v in SCSCF_ADDR_REG_DICT.items():
        item_df = temp_df[temp_df['scscfaddr'].str.contains(v)]
        item_dict = {}
        for scscfaddr, item in item_df.groupby('scscfaddr'):
            item_dict[scscfaddr] = item.index.to_list()
        scscfaddr_dict[k] = item_dict

    return scscfaddr_dict

In [60]:
get_scscfaddr_dict(df5)

{'zte': {'cqscscf5bzx.cq': [8615086637082,
   8615723666619,
   8613996765557,
   8615723198155],
  'cqscscf6bzx.cq': [8615213616672, 8615025593512, 8615826267293],
  'cqscscf7bzx.cq': [8615823274180,
   8615213752090,
   8615025648392,
   8615823083710,
   8618423394515,
   8615213358922],
  'cqscscf8bzx.cq': [8615223769474,
   8613647647595,
   8615023603533,
   8613709468964,
   8613628267979,
   8615823941201,
   8615023525354,
   8615202393237,
   8613896304363],
  'cqscscf9bzx.cq': [8615025451665,
   8618723314386,
   8618723397033,
   8615025473084,
   8618723583873,
   8615213053636]},
 'hw': {'cqscscf11bhw.cq': [8613527395750,
   8615923504757,
   8618223290274,
   8618203058190,
   8615923589669],
  'cqscscf12bhw.cq': [8613983975808,
   8613908323969,
   8618896096106,
   8613637741523,
   8613983908520,
   8613983075148,
   8613752814268],
  'cqscscf13bhw.cq': [8615923157014, 8618883738502],
  'cqscscf14bhw.cq': [8618896159969,
   8617848607458,
   8613608383070,
   86135273